In [210]:
import pandas as pd
import nltk
import numpy as np

#load inthe NTLK stopwords to remove articles, preposition and other words that are not actionable
from nltk.corpus import stopwords
# This allows to create individual objects from a bog of words
from nltk.tokenize import word_tokenize, sent_tokenize
# Lemmatizer helps to reduce words to the base form
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [211]:
df=pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/jasmineli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [212]:
def process_sentence(sentence):
    new_tokens = word_tokenize(sentence)
    new_tokens = [t.lower() for t in new_tokens]
    new_tokens =[t for t in new_tokens if t not in stopwords.words('english')]
    new_tokens = [t for t in new_tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(t) for t in new_tokens]
    return "".join(new_tokens)

In [213]:
titles = df['title_orig'].tolist()
tokens = [process_sentence(t) for t in titles]
df['title_pre'] = tokens

In [214]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['title_pre'])

In [215]:
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
tfidf = pd.DataFrame(denselist, columns=feature_names)

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [216]:
print(tfidf.shape)

(1573, 1159)


In [217]:
# product color
def main_color(s):
    main_color = {"red":"red", "white":"white", "pink":"pink", "yellow":"yellow", "green":"green", "blue":"blue", "wine":"red", "burgundy":"red", "black":"black", "navy":"navy", "orange":"orange", 
    "rose":"pink", "gray":"gray", "grey":"gray", "purple":"purple", "violet":"purple", "army":"green", "leopard":"orange", "ivory":"white", 
    "brown":"brown", "coffee":"brown", "camel":"beige", "tan":"brown", "nude":"beige", "khaki":"khaki", "apricot":"yellow", "camouflage":"green", "jasper":"red"}  # ordered by importance
    for key, value in main_color.items():
        if key in s:
            return value
    return "others"
product_color = df["product_color"]
product_color = [s.lower() if type(s) is str else 'nan' for s in product_color]
product_color = [main_color(s) for s in product_color]
from matplotlib import colors
product_color = [(-0.1,-0.1,-0.1,-0.1) if s == "others" else colors.to_rgba(s) for s in product_color]

df['product_color_rgb'] = [np.array(t) for t in product_color]

# log prices
df['log_price'] = [np.log(p) for p in df["price"]]
df['log_retail_price'] = [np.log(p) for p in df["retail_price"]]

# log merchant rating count
df['log_merchant_rating_count'] = np.log(df['merchant_rating_count'])

# urgent text
df['urgent'] = [1 if s == "Quantité limitée !" else 0 for s in df["urgency_text"]]

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [229]:
data = df[["log_price", "log_retail_price", "uses_ad_boosts", "badges_count", "badge_local_product", 
           "badge_product_quality", "badge_fast_shipping", "urgent", "units_sold"]]
df2 = pd.concat([data, tfidf], axis=1)
label = [1 if sales > 1000 else 0 for sales in data["units_sold"]]
df2['high_sale'] = label

In [219]:
rgb = df["product_color_rgb"]
rgb = np.stack(rgb.values, axis=0)
for i in range(4):
    df2["product_color_rgb"+str(i)] = rgb[:,i]
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
print(df2.shape)

(1573, 1173)


In [220]:
X = df2.loc[:, ~df2.columns.isin(['high_sale', 'units_sold'])]
# X = tfidf
y = df2['high_sale']

In [221]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.14286, random_state=42)

In [222]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of dev examples = " + str(X_dev.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(y_dev.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 1179
number of dev examples = 197
number of test examples = 197
X_train shape: (1179, 1171)
Y_train shape: (1179,)
X_dev shape: (197, 1171)
Y_dev shape: (197,)
X_test shape: (197, 1171)
Y_test shape: (197,)


In [223]:
# Defining hold out data for evaluation 
X_train = tf.expand_dims(X_train, axis=-1)
input_shape = X_train.shape[1:]

In [224]:
print(input_shape)

(1171, 1)


In [225]:
# from sklearn.linear_model import LogisticRegression

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# score = classifier.score(evals_X, evals_y)

# print("Accuracy:", score)

In [226]:
# define model
model = Sequential()
model.add(Conv1D(filters=24, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dense(1, activation='sigmoid'))

In [227]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [228]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    verbose=2,
    validation_data=(X_dev, y_dev),
    batch_size=32,
    callbacks=[es]
)
model.summary()

Epoch 1/1000
37/37 - 1s - loss: 1.1274 - accuracy: 0.5818 - val_loss: 0.6975 - val_accuracy: 0.6345 - 1s/epoch - 38ms/step


/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Epoch 2/1000
37/37 - 1s - loss: 0.6967 - accuracy: 0.5869 - val_loss: 0.6732 - val_accuracy: 0.6345 - 836ms/epoch - 23ms/step
Epoch 3/1000
37/37 - 1s - loss: 0.6938 - accuracy: 0.5869 - val_loss: 0.6742 - val_accuracy: 0.6396 - 889ms/epoch - 24ms/step
Epoch 4/1000
37/37 - 1s - loss: 0.6906 - accuracy: 0.5937 - val_loss: 0.6792 - val_accuracy: 0.6599 - 864ms/epoch - 23ms/step
Epoch 5/1000
37/37 - 1s - loss: 0.6934 - accuracy: 0.6031 - val_loss: 0.6870 - val_accuracy: 0.6701 - 844ms/epoch - 23ms/step
Epoch 6/1000
37/37 - 1s - loss: 0.6957 - accuracy: 0.6073 - val_loss: 0.6753 - val_accuracy: 0.6904 - 830ms/epoch - 22ms/step
Epoch 7/1000
37/37 - 1s - loss: 0.6958 - accuracy: 0.6098 - val_loss: 0.6815 - val_accuracy: 0.6548 - 866ms/epoch - 23ms/step
Epoch 8/1000
37/37 - 1s - loss: 0.6983 - accuracy: 0.6268 - val_loss: 0.6824 - val_accuracy: 0.6853 - 810ms/epoch - 22ms/step
Epoch 9/1000
37/37 - 1s - loss: 0.7029 - accuracy: 0.6243 - val_loss: 0.6937 - val_accuracy: 0.6802 - 850ms/epoch - 23